In [1]:
import pandas as pd
import sweetviz as sv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import itertools
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
df_clean_data = pd.read_csv('./csv/0_clean_data.csv')
df_clean_data = df_clean_data[df_clean_data['order_state']!=2]
df_clean_data

,customer_id,order_product_id,browser,platform,customer_firstname,customer_lastname,customer_email,customer_gender,customer_nationality,customer_country,...,departure_month,departure_day,departure_DayofYear,departure_DayofWeek,country_code,sum_kids,sum_adults,private,group,family
0,0,121,2,0,Julie,Staplea,juliestaples40@gmail.com,2,british,77,...,10,29,302,0,77,0,2,1,0,0
3,0,90,7,6,Dale,Lewis,juleslewis@hotmail.co.uk,0,welsh,77,...,8,31,243,4,77,0,2,1,0,0
5,0,90,7,6,Tai,Nguyen,taimusicpro@gmail.com,0,australia,13,...,8,31,243,4,13,2,3,0,0,1
6,0,90,2,0,Charlotte,Brandom,charlotte.brandom@hotmail.co.uk,2,british,77,...,8,31,243,4,77,0,2,1,0,0
9,0,90,2,4,Amelia Melinda,Mujang,ameliamelindamujang@gmail.com,0,malaysian,158,...,9,4,247,1,158,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9240,0,34,2,4,Noeleen,Sonego,noeleens@cdmaust.com.au,1,australian,13,...,2,7,38,1,13,0,2,1,0,0
9243,0,47,7,6,Peter,Naegelein,peter.naegelein@gmx.de,0,german,158,...,1,18,18,2,158,1,2,0,0,1
9245,2824,122,2,4,Ann Mary,Baby,annmarybaby5@gmail.com,0,india,105,...,2,15,46,2,105,0,2,1,0,0
9247,0,195,2,0,Odmunkh,Ebkhbold,odmunkh@yahoo.com,0,mongolia,147,...,1,17,17,1,147,0,5,0,1,0


In [3]:
df_country_code_matrix = pd.read_csv('./csv/3_country_code_matrix.csv')
df_country_code_matrix

,country_code,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,2,1.857143,0.267415,0.087755,0.912245,0.420042,0.287089,0.292869,0.051326,0.052598,...,0.065133,0.079568,0.063860,0.048781,0.048781,0.065736,0.048781,0.048781,0.050164,0.051548
1,12,0.000000,0.448750,0.000000,1.000000,0.394029,0.394029,0.211942,0.065974,0.064487,...,0.064487,0.045935,0.045935,0.045935,0.045935,0.045935,0.045935,0.045935,0.064487,0.045935
2,13,2.803536,0.276558,0.105667,0.894333,0.419428,0.277050,0.303522,0.057351,0.053122,...,0.055846,0.078463,0.054452,0.049144,0.051028,0.070759,0.049144,0.049144,0.052314,0.051541
3,15,0.000000,0.269500,0.500000,0.500000,0.211942,0.211942,0.576117,0.050714,0.050714,...,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714
4,17,0.000000,0.130000,0.000000,1.000000,0.576117,0.211942,0.211942,0.050714,0.050714,...,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714,0.050714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,231,20.000000,0.172500,0.226190,0.773810,0.211942,0.211942,0.576117,0.050714,0.050714,...,0.050714,0.094285,0.050714,0.050714,0.050714,0.094285,0.050714,0.050714,0.050714,0.050714
91,233,2.315018,0.317439,0.030569,0.969431,0.489408,0.267969,0.242623,0.055440,0.056385,...,0.056505,0.072141,0.055081,0.048687,0.050921,0.057777,0.049006,0.048687,0.057764,0.049644
92,241,3.444444,0.179496,0.148148,0.851852,0.333333,0.333333,0.333333,0.049878,0.049878,...,0.049878,0.087171,0.049878,0.049878,0.049878,0.096853,0.049878,0.049878,0.049878,0.049878
93,248,11.402597,0.246991,0.054978,0.945022,0.457878,0.268696,0.273426,0.056171,0.056017,...,0.057944,0.085727,0.054976,0.048884,0.050016,0.060829,0.048884,0.048884,0.050980,0.053411


In [8]:
a = df_country_code_matrix[df_country_code_matrix['country_code']==999].shape[0]

In [9]:
a

0

In [10]:
def compute_similarity(df1, df2, index_col1, index_col2, data_cols):
    similarity_matrix = cosine_similarity(df1[data_cols], df2[data_cols])
    similarity_df = pd.DataFrame(similarity_matrix, columns=df2[index_col2], index=df1[index_col1])
    return similarity_df

def compute_distance(df1, df2, index_col1, index_col2, data_cols):
    distance_matrix = euclidean_distances(df1[data_cols], df2[data_cols])
    distance_df = pd.DataFrame(distance_matrix, columns=df2[index_col2], index=df1[index_col1])
    return distance_df

def recommend_products(similarity_df, customer_id, n=5):
    customer_scores = similarity_df.loc[customer_id].sort_values(ascending=False)
    top_n_products = customer_scores.index[:n].tolist()
    return top_n_products

In [11]:
df_product_matrix = pd.read_csv('./csv/3_product_matrix.csv')
df_product_matrix.tail()

similarity_cols = [
    # 'booked_days', 
    'order_price_paid', 
    'sum_kids','sum_adults', 
    # 'private', 'group', 'family', 
    'cat_1', 'cat_2', 'cat_3','cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10',
    'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17','cat_18'
    ]
df_item_item_similarity = compute_similarity(df_country_code_matrix, df_product_matrix, 'country_code', 'order_product_id', similarity_cols)
distance_cols = [
    'booked_days', 
    'order_price_paid', 
    'sum_kids','sum_adults', 
    'private', 'group', 'family', 
    'cat_1', 'cat_2', 'cat_3','cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', 'cat_10',
    'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17','cat_18'
    ]
df_item_item_distance = compute_distance(df_country_code_matrix, df_product_matrix, 'country_code', 'order_product_id', distance_cols)

In [12]:
def recommend_top_n(similarity_df, distance_df, customer_id, top_n=10):
    # need load df_full_matrix and len(cleanHistory_list) < 10
    top_30 = list(map(int,recommend_products(similarity_df, customer_id, n=30)))
    customer_scores = distance_df.loc[customer_id].sort_values(ascending=False).index.tolist()
    distance_list = list(map(int,customer_scores))
    recommend_ls = list(itertools.filterfalse(lambda x: x not in top_30, distance_list))[:top_n]
    return recommend_ls

In [13]:
recommend_top_n(df_item_item_similarity, df_item_item_distance, 22,5)

[54, 91, 68, 147, 82]